# Multi-Entity-to-Abstract Dataset Creation

## Purpose

This notebook builds on the previous method by grouping **multiple biomedical entities** from a single abstract into a single training input.

The goal is to simulate more complex summarization or generation tasks, where the model learns to produce biomedical text conditioned on **a set of related concepts**.

### Example:
- **Input (Prompt):** `"Summarize findings involving: immune checkpoint inhibitors, nonsmall cell lung cancer, overall survival"`
- **Target (Output):** Abstract text

This setup more closely mimics real-world applications like question answering or scientific report generation, where multiple key topics are involved.


!pip install jsonlines

In [ ]:
# Only if you are using Google Colab and want to retreive the data from your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

with open("/content/drive/MyDrive/biomedical_text_generation/data/enriched/abstracts_with_entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Total abstracts: {len(data)}")


In [ ]:
multi_entity_pairs = []

for entry in data:
    abstract = entry["abstract"]
    pmid = entry.get("pmid")
    entities = entry.get("entities", [])
    
    # Filter entities with at least 2 words to keep meaningful ones
    filtered = [e for e in entities if len(e.split()) >= 2]
    
    # Only keep if there are at least 2-3 meaningful entities
    if len(filtered) >= 2:
        prompt = "Summarize findings involving: " + ", ".join(filtered[:5])  # Optional: limit to top 5 entities
        multi_entity_pairs.append({
            "input": prompt,
            "output": abstract,
            "pmid": pmid,
            "entities_used": filtered[:5]
        })

print(f"Generated {len(multi_entity_pairs)} training pairs.")


In [ ]:
import os
import jsonlines

output_path = "/content/drive/MyDrive/biomedical_text_generation/data/training/summarization/"
os.makedirs(output_path, exist_ok=True)

with jsonlines.open(os.path.join(output_path, "multi_entity_to_abstract.jsonl"), mode="w") as writer:
    writer.write_all(multi_entity_pairs)

print("Saved multi-entity dataset.")
